In [ ]:
import numpy as np 
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm 
from sklearn.metrics import silhouette_score


In [ ]:
def data_retrieval(threshold = 0.65):
    data_path = "./Data_Providers/"
    df_providers_class_content = pd.read_csv(data_path + 'content_overview_new_202509.csv')
    df_providers_class_service = pd.read_csv(data_path + 'service_overview_new_202509.csv')
    df_providers_class_content = df_providers_class_content[df_providers_class_content.columns[0:-1]].dropna()
    df_providers_class_service = df_providers_class_service[df_providers_class_service.columns[0:-1]].dropna()
    print(df_providers_class_content.describe()), print(df_providers_class_service.describe())
    df_new = pd.DataFrame()

    row = []

    for i in range(len(df_providers_class_service)):
        for j in range(len(df_providers_class_content)):
            if df_providers_class_content["instance"][j][0:-1] in df_providers_class_service.loc[i, "server"] and df_providers_class_content["feed"][j] == df_providers_class_service["feed"][i] and df_providers_class_content["date"][j] == df_providers_class_service["date"][i]:
        
                # build row
                row = {
                "feed": df_providers_class_service.loc[i, "feed"],
                "server": df_providers_class_service.loc[i, "server"],
                "map_matching": df_providers_class_content.loc[j, "map_matching"],
                "exclusive": df_providers_class_content.loc[j, "exclusive"],
                "redundant": df_providers_class_content.loc[j, "redundant"],
                "accidents": df_providers_class_service.loc[i, "accidents"],
                "hazards": df_providers_class_service.loc[i, "hazards"],
                "closures": df_providers_class_service.loc[i, "closures"],
                "freshness": df_providers_class_service.loc[i, "freshness"],
                "contribution": df_providers_class_service.loc[i, "contribution"],
                "usability": df_providers_class_service.loc[i, "usability"],
                "availability": df_providers_class_service.loc[i, "availability"],

                }
                row = pd.DataFrame([row])
                # append row to dataframe
                df_new = pd.concat([df_new, row])
          
        print("cycle i:", i, " out of ", len(df_providers_class_service))

    merged_df = df_new
    df_complete = merged_df
    df_complete['freshness'] = (df_complete['freshness'] - df_complete['freshness'].min()) / (df_complete['freshness'].max() - df_complete['freshness'].min())
    df = df_complete[df_complete.columns[2:]]
    corr_matrix = df.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title('Correlation Heatmap')
    plt.savefig("corr_matrix.png")
    plt.show()

    # Extract attributes with absolute correlation above 0.65 (excluding self-correlation)
    high_corr_cols = set()
    
    for col in corr_matrix.columns:
        for idx in corr_matrix.index:
            if col != idx and abs(corr_matrix.loc[idx, col]) > threshold:
                high_corr_cols.add(col)
                high_corr_cols.add(idx)

    # Convert to list
    high_corr_cols = list(high_corr_cols)
    print("Attributes with absolute correlation above 0.65:", high_corr_cols)

    # Optional: store these attributes in a new dataframe
    df_high_corr = df[high_corr_cols]

    df_subset = df

    # Plot the average value for each column
    df_subset.mean().plot(kind='bar', figsize=(12, 6))
    plt.ylabel('Average value')
    plt.title('Average Scores per Attribute')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("avg_attributes.png")
    plt.show()

    return df_subset, df_high_corr, df_subset.describe()

In [ ]:
from sklearn.mixture import GaussianMixture

def cluster_gmm(df, a=1,b=1.960, grades = ['A+', 'A', 'B', 'C', 'D']):

    df = df[['map_matching', 'exclusive', 'redundant', 'accidents',
       'hazards', 'closures', 'freshness', 'contribution', 'usability',
       'availability']]

    # --- Fit GMM ---
    gmm = GaussianMixture(n_components=len(grades), covariance_type='full', random_state=42)
    df['cluster_GMM'] = gmm.fit_predict(df)

    # --- Probabilities for each cluster ---
    proba = gmm.predict_proba(df.drop(columns=['cluster_GMM']))
    df[[f'cluster_{i}_prob' for i in range(5)]] = proba

    # --- Cluster summary
    cluster_summary = df.groupby('cluster_GMM')[['map_matching', 'redundant', 'freshness','exclusive',
                                         'availability', 'contribution', 'usability']].agg(['mean', 'min', 'max', 'count'])

    mean_cols = cluster_summary.loc[:, pd.IndexSlice[:, 'mean']]
    mean_cols.columns = mean_cols.columns.droplevel(1)

    # Average mean score
    cluster_summary['average_mean'] = mean_cols.mean(axis=1)

    # Variance score
    cluster_variance = df.groupby('cluster_GMM')[['map_matching', 'redundant', 'exclusive',
                                          'availability', 'freshness', 'contribution', 'usability']].var()
    cluster_summary['variance_score'] = cluster_variance.mean(axis=1)

    # --- Extract means for custom scoring ---
    map_matching_mean = cluster_summary[('map_matching', 'mean')]
    exclusive_mean = cluster_summary[('exclusive', 'mean')]
    redundant_mean = cluster_summary[('redundant', 'mean')]
    availability_mean = cluster_summary[('availability', 'mean')]
    freshness_mean = cluster_summary[('freshness', 'mean')]
    contribution_mean = cluster_summary[('contribution', 'mean')]
    usability_mean = cluster_summary[('usability', 'mean')]

    # --- Custom score formula ---
    cluster_summary['custom_score'] = (
        map_matching_mean + exclusive_mean + availability_mean +
        freshness_mean + contribution_mean + usability_mean - redundant_mean    
    )

    # Rank clusters by custom score
    cluster_summary['custom_rank'] = cluster_summary['custom_score'].rank(ascending=False)
    cluster_summary_mean = cluster_summary.sort_values('custom_rank', ascending=True)

    # Assign grades (custom score)
    
    cluster_summary_mean['grade'] = pd.Series(grades[:len(cluster_summary_mean)],
                                          index=cluster_summary_mean.index)

    # --- Rank by variability ---
    cluster_summary_var = cluster_summary.sort_values('variance_score', ascending=True)
    cluster_summary_var['grade'] = pd.Series(grades[:len(cluster_summary_var)],
                                         index=cluster_summary_var.index)

    rows_lb = []
    rows_ub = []
    for i in range(len(grades)):
      rows_lb.append(a * cluster_summary_mean.custom_score[i] - b * np.sqrt(cluster_summary_var.variance_score[i]) )
    cluster_summary['threshold_lb'] = rows_lb

    for i in range(len(grades)):
      rows_ub.append(a * cluster_summary_mean.custom_score[i] + b * np.sqrt(cluster_summary_var.variance_score[i]) )
    cluster_summary['threshold_ub'] = rows_ub

    # Rank by threshold
    cluster_summary_thld = cluster_summary.sort_values('threshold_ub', ascending=False)
    cluster_summary_thld['grade'] = pd.Series(grades[:len(cluster_summary_thld)],
                                          index=cluster_summary_thld.index)

    score_pca = silhouette_score(df, df['cluster_GMM'])
    bic = gmm.bic(df[['map_matching', 'exclusive', 'redundant', 'accidents',
       'hazards', 'closures', 'freshness', 'contribution', 'usability',
       'availability']])
    aic = gmm.aic(df[['map_matching', 'exclusive', 'redundant', 'accidents',
       'hazards', 'closures', 'freshness', 'contribution', 'usability',
       'availability']])
    
    return df, cluster_summary_mean[['custom_score', 'grade']], cluster_summary_var[['variance_score', 'grade']], cluster_summary_thld[['threshold_lb','threshold_ub', 'grade']].sort_index(), score_pca, bic,aic

In [ ]:
def cluster_kmeans(df, a=1,b=1.960,  grades = ['A+', 'A', 'B', 'C', 'D'] ):

    kmeans = KMeans(n_clusters=5, random_state=42)
    df['cluster'] = kmeans.fit_predict(df)

    cluster_summary = df.groupby('cluster')[['map_matching', 'redundant', 'freshness','exclusive', 'availability', 'contribution', 'usability']].agg(['mean', 'min', 'max', 'count'])
    mean_cols = cluster_summary.loc[:, pd.IndexSlice[:, 'mean']]

    # Rename columns for easier access (optional)
    mean_cols.columns = mean_cols.columns.droplevel(1)  # drop second level 'mean'

    # Now calculate average of the three feature means for each cluster
    cluster_summary['average_mean'] = mean_cols.mean(axis=1)

    cluster_variance = df.groupby('cluster')[['map_matching', 'redundant', 'exclusive', 'availability', 'freshness', 'contribution', 'usability']].var()
    cluster_summary['variance_score'] = cluster_variance.mean(axis=1)

    # Extract individual mean values
    map_matching_mean = cluster_summary[('map_matching', 'mean')]
    exclusive_mean = cluster_summary[('exclusive', 'mean')]
    redundant_mean = cluster_summary[('redundant', 'mean')]
    availability_mean = cluster_summary[('availability', 'mean')]
    freshness_mean = cluster_summary[('freshness', 'mean')]
    contribution_mean = cluster_summary[('contribution', 'mean')]
    usability_mean = cluster_summary[('usability', 'mean')]
    # Apply the custom scoring formula
    cluster_summary['custom_score'] = map_matching_mean + exclusive_mean + availability_mean + freshness_mean + contribution_mean + usability_mean - redundant_mean

    # Rank clusters by custom score
    cluster_summary['custom_rank'] = cluster_summary['custom_score'].rank(ascending=False)

    # Sort clusters by average_mean descending (best quality first)
    cluster_summary_mean = cluster_summary.sort_values('custom_rank', ascending=True)

    # Define grades to assign by rank
 

    # Assign grades based on rank position
    cluster_summary_mean['grade'] = pd.Series(grades[:len(cluster_summary_mean)], index=cluster_summary_mean.index)
    cluster_summary_mean[['custom_score', 'grade']]

    # Sort clusters by variability ascending (best quality first)
    cluster_summary_var = cluster_summary.sort_values('variance_score', ascending=True)

    # Define grades to assign by rank
  

    # Assign grades based on rank position
    cluster_summary_var['grade'] = pd.Series(grades[:len(cluster_summary_var)], index=cluster_summary_var.index)
    cluster_summary_var[['variance_score', 'grade']]
    cluster_summary['threshold'] = np.zeros(5)
    rows_lb = []
    rows_ub = []
    for i in range(5):
        rows_lb.append(a * cluster_summary_mean.custom_score[i] - b * np.sqrt(cluster_summary_var.variance_score[i]) )
    cluster_summary['threshold_lb'] = rows_lb

    for i in range(5):
        rows_ub.append(a * cluster_summary_mean.custom_score[i] + b * np.sqrt(cluster_summary_var.variance_score[i]) )
    cluster_summary['threshold_ub'] = rows_ub

    # Sort clusters by variability ascending (best quality first)
    cluster_summary_thld= cluster_summary.sort_values('threshold_ub', ascending=False)
  

    # Assign grades based on rank position
    cluster_summary_thld['grade'] = pd.Series(grades[:len(cluster_summary_thld)], index=cluster_summary_thld.index)

    score_pca = silhouette_score(df, df['cluster'])
    
    return df, cluster_summary_mean[['custom_score', 'grade']], cluster_summary_var[['variance_score', 'grade']], cluster_summary_thld[['threshold_lb', 'threshold_ub','grade']].sort_index(), score_pca


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D  # Needed even if not directly used

def plot(df, var_name):


    fig = plt.figure(figsize=(18, 6))  # Wider layout for side-by-side

    # Define your triplets and titles
    plot_configs = [
    ('accidents', 'hazards', 'closures', 'Incident Coverage'),
    ('exclusive', 'redundant', 'contribution', 'Data Richness'),
    ('freshness', 'availability', 'map_matching', 'Service Performance')]

    # Loop to create subplots
    for idx, (x_col, y_col, z_col, title) in enumerate(plot_configs, start=1):
        ax = fig.add_subplot(1, 3, idx, projection='3d')
    
        scatter = ax.scatter(
            df[x_col],
            df[y_col],
            df[z_col],
            c=df[var_name],
            cmap='tab10',
            s=50,
            alpha=0.8
        )
    
        ax.set_xlabel(x_col)
        ax.set_ylabel(y_col)
        ax.set_zlabel(z_col)

        # Apply consistent inversion logic if needed (edit per axis logic)
        ax.invert_xaxis()
        ax.invert_yaxis()
        ax.invert_zaxis()
    
        ax.set_title(title)

    # Shared legend (outside of plot area)
    handles = [
        Line2D([0], [0], marker='o', color='w', label=f'Cluster {i}',
           markerfacecolor=scatter.cmap(scatter.norm(i)), markersize=8)
        for i in sorted(df[var_name].unique())
    ]

    fig.legend(handles=handles, title='Clusters', loc='center right', bbox_to_anchor=(1.05, 0.5))
    fig.suptitle('3D Scatter Plots by Dimension Group', fontsize=16)
    plt.tight_layout(rect=[0, 0, 0.88, 1])  # shrink plot area slightly more
    fig.subplots_adjust(wspace=0.3)        # increase horizontal space between subplots

    plt.show()


In [ ]:
def market_requirements():

    market_req = pd.DataFrame({"Accuracy_lb": [0, 0.15, 0.5,0.85,0.95],"Completeness_lb": [0,0.10,0.4,0.6,0.90],"Correctness_lb": [0,0.15,0.5,0.85,0.95], "Availability_lb": [0,0.85,0.95,0.97,0.99], "Timelineness_lb": [0,0.15,0.8,0.85,0.95], "Usability_lb": [0,0.15,0.5,0.85,0.95] , 
                              "Accuracy_ub": [0.15, 0.50, 0.85,0.95,1],"Completeness_ub": [0.1,0.4,0.6,0.9,1],"Correctness_ub": [0.15,0.5,0.85,0.95,1], "Availability_ub": [0.85,0.95,0.97,0.99,1], "Timelineness_ub": [0.15,0.8,0.85,0.95,1], "Usability_ub": [0.15,0.5,0.85,0.95,1] })
    market_req['Classification'] = ["D", "C", "B", "A", "A+"]
    
    return market_req

In [ ]:
df, corr, df_describe = data_retrieval()

In [ ]:
df_clus, df_clus_mean, df_clus_var, df_clus_thld, score_gmm, bic, aic =  cluster_gmm(df)
df_kmean, df_kmean_mean,df_kmean_var, df_kmeans_thld, kmean_metric = cluster_kmeans(df)


In [ ]:
if score_gmm > kmean_metric:
    var_name = 'cluster_GMM'
    plot(df_clus, var_name)
    print ("Silhouette score: " , score_gmm)

else:
    var_name = 'cluster'
    plot(df, var_name)
    print ("Silhouette score: " , kmean_metric)

In [ ]:
market_req = market_requirements()
market_req[market_req.columns[0]][market_req['Classification']=='A'] , market_req[market_req.columns[6]][market_req['Classification']=='A']

In [ ]:
df_clus_thld

In [ ]:
df_kmeans_thld